# Chronos 2 Time Series Prediction Playbook

[Huggingface](https://huggingface.co/amazon/chronos-2)

In [ ]:
def extend_path():
    """Extend notebooks system path config to import relative packages."""
    import sys
    from pathlib import Path

    parent_folder = str(Path.cwd().parent)
    print(f"Adding {parent_folder} to sys.path")
    if parent_folder not in sys.path:
        sys.path.insert(0, parent_folder)


extend_path()

In [ ]:
import torch
from chronos import Chronos2Pipeline
from playground.utils import plot_forecast

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

MODEL_PATH = "amazon/chronos-2"
N_TIMESERIES = 1
N_VARIATES = 1

In [ ]:
pipeline: Chronos2Pipeline = Chronos2Pipeline.from_pretrained(
    pretrained_model_name_or_path=MODEL_PATH,
    device_map=DEVICE,  # use "cpu" for CPU inference and "mps" for Apple Silicon
    dtype=torch.bfloat16,
)

In [ ]:
# A 3-dimensional torch.Tensor or np.ndarray of shape (batch, n_variates, history_length).
# When n_variates > 1, information will be shared among the different variates of each
# time series in the batch and the model will perform multivariate forecasting.
torch.manual_seed(42)
context = torch.rand((N_TIMESERIES, N_VARIATES, 128), device=DEVICE)
context.shape

In [ ]:
forecast: list[torch.Tensor] = pipeline.predict(inputs=context, prediction_length=64)

In [ ]:
quantiles = forecast[0].permute(0, 2, 1)

In [ ]:
quantiles[0, 0, :]

In [ ]:
plot_forecast(
    ctx=context.squeeze(0), quantile_fc=quantiles, real_future_values=None, batch_idx=0
)